In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

def preprocess_air_quality_data(file_path):
    # Load data
    df = pd.read_csv(file_path, low_memory=False)
    df.drop(columns=['City'], inplace=True)
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=['float64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
    
    # Process datetime and create time features
    df['Datetime'] = pd.to_datetime(df['Datetime'], format='mixed', errors='coerce')
    df['Year'] = df['Datetime'].dt.year
    df['Month'] = df['Datetime'].dt.month
    df['Day'] = df['Datetime'].dt.day
    df['Hour'] = df['Datetime'].dt.hour
    
    # Categorize PM2.5 values
    bins = [0, 12, 35.4, 55.4, 150.4, 250.4, float('inf')]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['PM2.5_Category'] = pd.cut(df['PM2.5'], bins=bins, labels=labels)
    
    # Perform undersampling
    return df


# For each dataset
df = preprocess_air_quality_data('../Data/Ernakulam_Guwahati_Jaipur_data.csv')

# Print sizes to see the reduction
print("Original sizes:", len(df))

Original sizes: 42559


In [3]:
df.head()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Year,Month,Day,Hour,PM2.5_Category
0,2020-01-22 13:00:00,33.830000,78.250000,130.280000,7.630000,137.910000,103.770000,2.020000,11.443253,40.07494,0.500000,6.13834,0.000000,132.425958,NaN,2020,1,22,13,Moderate
1,2020-01-22 14:00:00,41.000000,87.000000,128.780000,7.770000,136.520000,102.780000,2.100000,11.443253,40.07494,0.870000,6.13834,0.210000,132.425958,NaN,2020,1,22,14,Unhealthy for Sensitive
2,2020-01-22 15:00:00,54.242805,114.797399,16.927518,25.415553,39.683455,21.545806,0.855691,11.443253,40.07494,1.778407,6.13834,0.157817,132.425958,NaN,2020,1,22,15,Unhealthy for Sensitive
3,2020-01-22 16:00:00,40.100000,83.360000,125.700000,7.900000,133.620000,100.350000,1.780000,3.350000,40.07494,0.530000,0.72000,0.000000,132.425958,NaN,2020,1,22,16,Unhealthy for Sensitive
4,2020-01-22 17:00:00,35.000000,74.000000,126.470000,7.760000,134.240000,100.990000,1.840000,3.330000,40.07494,0.670000,1.30000,0.290000,132.425958,NaN,2020,1,22,17,Moderate


In [4]:
df.describe()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,Year,Month,Day,Hour
count,42559,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000,42559.000000
mean,2019-05-01 03:23:38.379191296,54.242805,114.797399,16.927518,25.415553,39.683455,21.545806,0.855691,11.443253,40.074940,1.778407,6.138340,0.157817,132.425958,2018.863672,6.101882,15.892596,11.502643
min,2017-06-14 08:00:00,0.060000,0.080000,0.010000,0.010000,0.000000,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,16.000000,2017.000000,1.000000,1.000000,0.000000
25%,2018-08-31 15:30:00,27.150000,56.840000,4.350000,11.470000,16.900000,11.540000,0.450000,8.310000,18.240000,0.170000,2.130000,0.157817,84.000000,2018.000000,3.000000,8.000000,6.000000
50%,2019-07-03 23:00:00,44.530000,98.390000,7.940000,22.330000,29.250000,19.230000,0.700000,11.140000,37.090000,0.650000,6.138340,0.157817,114.000000,2019.000000,6.000000,16.000000,12.000000
75%,2020-02-04 06:00:00,64.770000,141.725000,15.620000,32.230000,42.610000,26.180000,1.040000,14.040000,53.875000,1.690000,6.138340,0.157817,160.000000,2020.000000,9.000000,24.000000,18.000000
max,2020-07-01 00:00:00,923.080000,1000.000000,347.100000,380.020000,476.650000,305.640000,7.870000,140.840000,199.670000,491.510000,157.140000,17.760000,1109.000000,2020.000000,12.000000,31.000000,23.000000
std,NaN,49.344891,95.817353,26.780920,20.328262,40.072637,16.928914,0.662312,5.790171,27.752716,10.273706,6.460794,0.219133,76.665284,0.967334,3.318418,8.798025,6.922631


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42559 entries, 0 to 42558
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Datetime        42559 non-null  datetime64[ns]
 1   PM2.5           42559 non-null  float64       
 2   PM10            42559 non-null  float64       
 3   NO              42559 non-null  float64       
 4   NO2             42559 non-null  float64       
 5   NOx             42559 non-null  float64       
 6   NH3             42559 non-null  float64       
 7   CO              42559 non-null  float64       
 8   SO2             42559 non-null  float64       
 9   O3              42559 non-null  float64       
 10  Benzene         42559 non-null  float64       
 11  Toluene         42559 non-null  float64       
 12  Xylene          42559 non-null  float64       
 13  AQI             42559 non-null  float64       
 14  AQI_Bucket      41105 non-null  object        
 15  Ye

In [6]:
df.isna().sum()

Datetime             0
PM2.5                0
PM10                 0
NO                   0
NO2                  0
NOx                  0
NH3                  0
CO                   0
SO2                  0
O3                   0
Benzene              0
Toluene              0
Xylene               0
AQI                  0
AQI_Bucket        1454
Year                 0
Month                0
Day                  0
Hour                 0
PM2.5_Category       0
dtype: int64

In [7]:
df_tmp = df.copy()

In [8]:
df_tmp.drop(['AQI_Bucket', 'Datetime'], axis=1, inplace=True)
df_tmp.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,Year,Month,Day,Hour,PM2.5_Category
0,33.830000,78.250000,130.280000,7.630000,137.910000,103.770000,2.020000,11.443253,40.07494,0.500000,6.13834,0.000000,132.425958,2020,1,22,13,Moderate
1,41.000000,87.000000,128.780000,7.770000,136.520000,102.780000,2.100000,11.443253,40.07494,0.870000,6.13834,0.210000,132.425958,2020,1,22,14,Unhealthy for Sensitive
2,54.242805,114.797399,16.927518,25.415553,39.683455,21.545806,0.855691,11.443253,40.07494,1.778407,6.13834,0.157817,132.425958,2020,1,22,15,Unhealthy for Sensitive
3,40.100000,83.360000,125.700000,7.900000,133.620000,100.350000,1.780000,3.350000,40.07494,0.530000,0.72000,0.000000,132.425958,2020,1,22,16,Unhealthy for Sensitive
4,35.000000,74.000000,126.470000,7.760000,134.240000,100.990000,1.840000,3.330000,40.07494,0.670000,1.30000,0.290000,132.425958,2020,1,22,17,Moderate


In [9]:
df_tmp['PM2.5_Category'] = df_tmp['PM2.5_Category'].astype("category").cat.as_ordered()

In [10]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42559 entries, 0 to 42558
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   PM2.5           42559 non-null  float64 
 1   PM10            42559 non-null  float64 
 2   NO              42559 non-null  float64 
 3   NO2             42559 non-null  float64 
 4   NOx             42559 non-null  float64 
 5   NH3             42559 non-null  float64 
 6   CO              42559 non-null  float64 
 7   SO2             42559 non-null  float64 
 8   O3              42559 non-null  float64 
 9   Benzene         42559 non-null  float64 
 10  Toluene         42559 non-null  float64 
 11  Xylene          42559 non-null  float64 
 12  AQI             42559 non-null  float64 
 13  Year            42559 non-null  int32   
 14  Month           42559 non-null  int32   
 15  Day             42559 non-null  int32   
 16  Hour            42559 non-null  int32   
 17  PM2.5_Catego

In [11]:
df_tmp['PM2.5_Category'].cat.categories

Index(['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy',
       'Very Unhealthy', 'Hazardous'],
      dtype='object')

In [12]:
df_tmp['PM2.5_Category'].cat.codes

0        1
1        2
2        2
3        2
4        1
        ..
42554    2
42555    2
42556    2
42557    2
42558    3
Length: 42559, dtype: int8

In [13]:
df_tmp.isnull().sum()/len(df_tmp)

PM2.5             0.0
PM10              0.0
NO                0.0
NO2               0.0
NOx               0.0
NH3               0.0
CO                0.0
SO2               0.0
O3                0.0
Benzene           0.0
Toluene           0.0
Xylene            0.0
AQI               0.0
Year              0.0
Month             0.0
Day               0.0
Hour              0.0
PM2.5_Category    0.0
dtype: float64

In [14]:
np.random.seed(42)
from sklearn.model_selection import train_test_split

X = df_tmp.drop("PM2.5", axis=1)
Y = df_tmp["PM2.5"]

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8)

len(X_train), len(X_valid)

X_train.shape, X_valid.shape

((34047, 17), (8512, 17))

In [15]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)


PM10
NO
NO2
NOx
NH3
CO
SO2
O3
Benzene
Toluene
Xylene
AQI
Year
Month
Day
Hour


In [16]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label, "--> has", pd.isnull(content).sum(), "null values")

In [17]:
pd.Categorical(X_train["PM2.5_Category"]).codes

array([2, 0, 3, ..., 3, 3, 1], dtype=int8)

In [18]:
for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        #turn category into number
        X_train[label] = pd.Categorical(content).codes + 1

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34047 entries, 8729 to 15795
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PM10            34047 non-null  float64
 1   NO              34047 non-null  float64
 2   NO2             34047 non-null  float64
 3   NOx             34047 non-null  float64
 4   NH3             34047 non-null  float64
 5   CO              34047 non-null  float64
 6   SO2             34047 non-null  float64
 7   O3              34047 non-null  float64
 8   Benzene         34047 non-null  float64
 9   Toluene         34047 non-null  float64
 10  Xylene          34047 non-null  float64
 11  AQI             34047 non-null  float64
 12  Year            34047 non-null  int32  
 13  Month           34047 non-null  int32  
 14  Day             34047 non-null  int32  
 15  Hour            34047 non-null  int32  
 16  PM2.5_Category  34047 non-null  int8   
dtypes: float64(12), int32(4), int8(1)

In [20]:
for label, content in X_valid.items():
    if not pd.api.types.is_numeric_dtype(content):
        X_valid[label] = pd.Categorical(content).codes + 1

In [21]:
X_train.shape, X_valid.shape

((34047, 17), (8512, 17))

In [22]:
%%time
# Lets build a machine learning model
from sklearn.ensemble import RandomForestRegressor

CPU times: total: 78.1 ms
Wall time: 79.9 ms


In [23]:
np.random.seed(42)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    calculate root mean square log error between prediction and true labels    
    """

    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# create a function to evaluate model on a few different levels

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE" : mean_absolute_error(Y_train,train_preds),
              "valid MAE" : mean_absolute_error(Y_valid, val_preds),
              "Training RMSLE" : rmsle(Y_train,train_preds),
              "valid RMSLE" : rmsle(Y_valid, val_preds),
              "Training R^2": r2_score(Y_train,train_preds),
              "valid R^2" : r2_score(Y_valid, val_preds)}
    
    return scores

In [24]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [25]:
%%time
model.fit(X_train, Y_train)

CPU times: total: 1min 14s
Wall time: 8.08 s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [26]:
show_scores(model)

{'Training MAE': 2.031931440336567,
 'valid MAE': 5.2616221546564255,
 'Training RMSLE': 0.06535256626104778,
 'valid RMSLE': 0.15796680443669303,
 'Training R^2': 0.9940228135643917,
 'valid R^2': 0.9593214211922949}

In [27]:
%%time

# most ideal parameter 

ideal_model = RandomForestRegressor(n_estimators=90,
                                    min_samples_leaf=18,
                                    min_samples_split=3,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42)  # random state so that our reult is reproducable

# FIT THE MODEL

ideal_model.fit(X_train, Y_train)

CPU times: total: 17.8 s
Wall time: 2.02 s


RandomForestRegressor(max_features=0.5, min_samples_leaf=18,
                      min_samples_split=3, n_estimators=90, n_jobs=-1,
                      random_state=42)

In [28]:
show_scores(ideal_model)

{'Training MAE': 5.390598571465053,
 'valid MAE': 5.934493497722609,
 'Training RMSLE': 0.15583225459624933,
 'valid RMSLE': 0.17709593959234396,
 'Training R^2': 0.946687222734307,
 'valid R^2': 0.9361391998876227}

# Linear Regression

In [29]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,Y_train)

LinearRegression()

In [30]:
y_pred = linreg.predict(X_valid)

In [31]:
from sklearn.model_selection import cross_val_score
print('Cross_val_score', cross_val_score(linreg, X_train, Y_train, cv=10, scoring="r2").mean())
print('r2_score', r2_score(Y_valid, y_pred))
print("valid MAE ", mean_absolute_error(Y_valid, y_pred))

Cross_val_score 0.6846592316976261
r2_score 0.6967463252835266
valid MAE  13.46118538494203
